In [1]:
import pandas as pd
import joblib
import re
import ast

In [2]:
def make_new_dataset(df_raw):
    # отфильтруем возможные строковые «шапки»/комментарии
    df_raw = df_raw[df_raw["spans"].astype(str).str.strip().str.startswith("[")].reset_index(drop=True)
    
    rows = []
    for sent_id, row in df_raw.iterrows():
        text = str(row["text"]) if pd.notna(row["text"]) else ""
        spans_str = str(row["spans"])
    
        # парсим список спанов: [(start, end, 'B-TAG'), ...]
        try:
            entities = ast.literal_eval(spans_str)
        except Exception:
            # если строка поломана — пропускаем
            continue
    
        # токенизация: последовательности непробельных символов
        # сохраняем позицию каждого токена (start, end)
        token_spans = [(m.group(), m.start(), m.end()) for m in re.finditer(r"\S+", text)]
    
        # сопоставляем каждому токену метку по покрытию спаном
        for tok, s, e in token_spans:
            tag = "O"
            for es, ee, elabel in entities:
                # токен полностью внутри спана -> берём его метку
                if s >= es and e <= ee:
                    tag = elabel
                    break
            rows.append({"sent_id": sent_id, "data": tok, "entities": tag})

    # итоговая таблица «как на скриншоте»
    df = pd.DataFrame(rows, columns=["sent_id", "data", "entities"])
    return df

def convert_model2_to_model1(text: str, tags: list[str]):
    """
    Перевод предсказаний модели 2 (BIO список по токенам)
    в формат модели 1 [(start, end, tag), ...].
    """
    result = []
    tokens = text.split()
    offset = 0
    for token, tag in zip(tokens, tags):
        start = text.find(token, offset)  # ищем сдвигом, чтобы правильно найти индекс
        end = start + len(token)
        result.append((start, end, tag))
        offset = end
    return result

In [3]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit()
    }
    if i > 0:
        word1 = sent[i-1][0]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper()
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper()
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]

In [4]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s['data'].values.tolist(), 
                                                           s['entities'].values.tolist())]
        self.grouped = self.data.groupby('sent_id').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None

In [5]:
sub = pd.read_csv("submissions/sub_base.csv", sep=';')
sub.columns = ["text", "spans"]
crf = joblib.load('model/model_crf/crf_model.joblib')

sub_df = make_new_dataset(sub)
# пример вывода (первые 20 строк)
print(sub_df.head(20).to_string(index=False))

In [6]:
version = "ebeqshie_v5"

sub_getter = SentenceGetter(sub_df)
sub_sentences = sub_getter.sentences
sub_X = [sent2features(s) for s in sub_sentences]
y_pred_sub = crf.predict(sub_X)

new_tags = []
for i, tags in enumerate(y_pred_sub):
    text = sub.iloc[i]["text"]
    result = convert_model2_to_model1(text, tags)
    new_tags.append(result)
sub["annotation"] = new_tags
sub.rename(columns={"text": "sample"})[["sample", "annotation"]].to_csv(f"submissions/sub_{version}.csv", sep=';', index=False)